In [1]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime
from itertools import product
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

# 回测框架
import backtrader as bt

# 基础函数
import utilsJ

# 策略

In [2]:
class Strategy_comp1(bt.Strategy):
    
    params = (
        ('printlog', False),
        ('p_stake', 100),
        ('wave_period', 7),
        ('using_atr', False),
        ('atr_initial', 5000),
        ('atr_period', 14),
        ('atr_percent', 1),
        ('atr_risk', 1),
        ('bollinger_per', 20),
        ('bollinger_dev', 2),
        ('rsi_short', 6),
        ('rsi_long', 12),
        ('wr', 14),
        ('buy_signal', 'b'),
        ('sell_signal', 'b'),
    )
    
    
    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))
        
    
    def __init__(self):
        
        # Keep references to lines in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.datahigh = self.datas[0].high
        self.datalow = self.datas[0].low

        # Keep references to executed order & price
        self.buyprice = 0
        self.sellprice = 0
        self.order = None

        # Add indicators
        ## Wave indicator
        ### MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
                    self.datas[0], period=self.params.wave_period)
        
        ## Wave singal list

        self.wave_buy = bt.And(self.sma(0) > self.sma(-1), self.sma(-1) < self.sma(-2), self.sma(-2) < self.sma(-3))
        self.wave_sell = bt.And(self.sma(0) < self.sma(-1), self.sma(-1) > self.sma(-2), self.sma(-2) > self.sma(-3))

        ## MACD indicator
        self.macdhisto = bt.indicators.MACDHisto(self.datas[0])
        self.macd_buy = bt.And(self.macdhisto.histo(0) > 0, self.macdhisto.histo(-1) < 0)
        self.macd_sell = bt.And(self.macdhisto.histo(0) < 0, self.macdhisto.histo(-1) > 0)

        ## ATR indicator
        self.tr = bt.indicators.Max((self.datahigh - self.datalow), 
                                    abs(self.dataclose(-1) - self.datahigh),
                                    abs(self.dataclose(-1) - self.datalow))
        self.atr = bt.indicators.SimpleMovingAverage(self.tr, period=self.params.atr_period)
        
        ## Bollinger Bands indicator
        self.bolling_top = bt.indicators.BollingerBands(self.datas[0], 
                                                        period = self.params.bollinger_per, 
                                                        devfactor = self.params.bollinger_dev).top
        self.bolling_bot = bt.indicators.BollingerBands(self.datas[0], 
                                                        period = self.params.bollinger_per, 
                                                        devfactor = self.params.bollinger_dev).bot
        self.bolling_buy = bt.Or(bt.And(self.dataclose(0) > self.bolling_top(0), self.dataclose(-1) < self.bolling_top(-1)), 
                                 bt.And(self.dataclose(0) > self.bolling_bot(0), self.dataclose(-1) < self.bolling_bot(-1)))
        self.bolling_sell = bt.Or(bt.And(self.dataclose(0) < self.bolling_top(0), self.dataclose(-1) > self.bolling_top(-1)), 
                                 bt.And(self.dataclose(0) < self.bolling_bot(0), self.dataclose(-1) > self.bolling_bot(-1)))

        ## RSI indicator
        self.rsi_s = bt.indicators.RelativeStrengthIndex(period=self.params.rsi_short)
        self.rsi_l = bt.indicators.RelativeStrengthIndex(period=self.params.rsi_long)
        self.rsi_buy = bt.And(self.rsi_s(0) > self.rsi_l(0), self.rsi_s(-1) < self.rsi_l(-1))
        self.rsi_sell = bt.And(self.rsi_s(0) < self.rsi_l(0), self.rsi_s(-1) > self.rsi_l(-1))

        ## WilliamR indicator
        self.wr = bt.indicators.WilliamsR(period=self.params.wr)
        self.wr_buy = self.wr < -80
        self.wr_sell = self.wr > -20


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                if self.wave_buy:
                    self.buyprice = order.executed.price
                self.log('BUY EXECUTED, Price:%.2f, Lot:%i, Cash:%i.' %
                         (order.executed.price,
                          order.executed.size,
                          self.broker.get_cash()))

            else:  # Sell
                self.sellprice = order.executed.price
                self.log('SELL EXECUTED, Price:%.2f, Lot:%i, Cash:%i.' %
                        (order.executed.price,
                          -order.executed.size,
                          self.broker.get_cash()))

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        #self.order = None


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
    

    def next(self):
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        buy_indicator = (self.params.buy_signal == 'b' and self.bolling_buy) or \
                        (self.params.buy_signal == 'm' and self.macd_buy) or \
                        (self.params.buy_signal == 'w' and self.wave_buy and self.dataclose[0] > self.buyprice) or \
                        (self.params.buy_signal == 'r' and self.rsi_buy) or \
                        (self.params.buy_signal == 'wr' and self.wr_buy)
        sell_indicator = (self.params.sell_signal == 'b' and self.bolling_sell) or \
                         (self.params.sell_signal == 'm' and self.macd_sell) or \
                         (self.params.sell_signal == 'w' and self.wave_sell) or \
                         (self.params.sell_signal == 'r' and self.rsi_sell) or \
                         (self.params.sell_signal == 'wr' and self.wr_sell)
            
        if buy_indicator:
            if self.params.using_atr:
                opt_pos = np.round(self.params.atr_initial * self.params.atr_percent / self.params.p_stake / self.atr[0])
                if self.getposition(self.data).size == 0:
                    self.log('BUY CREATE, Price: %.2f, Lots: %i, Current Position: %i' % (self.dataclose[0], 
                        opt_pos*self.params.p_stake,
                        self.getposition(self.data).size))
                    self.buy(size=opt_pos*self.params.p_stake)
                elif self.getposition(self.data).size < opt_pos*self.params.p_stake:    
                    self.log('BUY CREATE, Price: %.2f, Lots: %i, Current Position: %i' % (self.dataclose[0], 
                            opt_pos*self.params.p_stake-self.getposition(self.data).size,
                            self.getposition(self.data).size))
                    self.buy(size=opt_pos*self.params.p_stake-self.getposition(self.data).size)
                elif self.getposition(self.data).size > opt_pos*self.params.p_stake:
                    self.log('SELL CREATE, Price: %.2f, Lots: %i, Current Position: %i' % (self.dataclose[0], 
                            self.getposition(self.data).size-opt_pos*self.params.p_stake,
                            self.getposition(self.data).size))
                    self.sell(size=self.getposition(self.data).size-opt_pos*self.params.p_stake)
            else:
                self.log('BUY CREATE, Price: %.2f, Lots: %i, Current Position: %i' % (self.dataclose[0], 
                        self.params.p_stake,
                        self.getposition(self.data).size))
                self.buy(size=self.params.p_stake)
        elif sell_indicator:
            self.log('Sell CREATE(Close), Price: %.2f, Current Position: %i' % (self.dataclose[0],
                        self.getposition(self.data).size))
            self.close()


    def stop(self):
        self.log('Ending Position:%i. Ending Value:%.2f.' %
                (self.getposition(self.data).size,
                self.cerebro.broker.getvalue()), doprint=False)

# 全市场选股

## 数据下载

In [31]:
s_date = datetime.date(2020, 12, 31) - datetime.timedelta(days = 365)
e_date = datetime.date(2020, 12, 31)

#s_date = datetime.datetime.now() - datetime.timedelta(days = 365)
#e_date = datetime.datetime.now()

stock_index = '399300.SZ'

# Download Data
utilsJ.index_to_csv_tushare('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6', stock_index,
                             0.2, s_date, e_date)

In [37]:
profit_list = []

# 904ff4752522814dca00e032a709fdfc26d8744913500ef204e02157
# 74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6

s_date = datetime.date(2020, 12, 31) - datetime.timedelta(days = 365)
e_date = datetime.date(2020, 12, 31)

pro = ts.pro_api('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6')
index_list = np.unique(pro.index_weight(index_code=stock_index,
                                        start_date=s_date.strftime('%Y%m%d'),
                                        end_date=e_date.strftime('%Y%m%d')).con_code).tolist()
idx = 0

for stock_c in index_list[0:300]:
    idx += 1
    buy_s = ['b','m','w', 'r', 'wr']
    sell_s = ['b','m','w', 'r', 'wr']
    profit_dict = {}
    for s in product(*[buy_s, sell_s]):
            
        # Create a cerebro entity
        cerebro = bt.Cerebro()

        # Add a strategy
        strats = cerebro.addstrategy(Strategy_comp1, printlog=False, using_atr=True, atr_percent=0.01, buy_signal = s[0], sell_signal = s[1]) 


        # Create stock Data Feed
        #df = utilsJ.stock_tushare('904ff4752522814dca00e032a709fdfc26d8744913500ef204e02157', 
        #                    stock_c, s_date, e_date)
        df = pd.read_csv('.\\Data\\' + stock_c + '.csv')
        df.index=pd.to_datetime(df.trade_date)
        data = bt.feeds.PandasData(dataname=df,fromdate=s_date,todate=e_date)

        # Add the index Data Feed to Cerebo
        cerebro.adddata(data)
        
        # Set cash inside the strategy
        cerebro.broker = bt.brokers.BackBroker(coc=True)   
        cerebro.broker.setcash(5000)

        # Set commission
        #cerebro.broker.setcommission()

        # Print out the starting conditions
        start_value = cerebro.broker.getvalue()
        #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

        # Run over everything
        cerebro.run()

        # Print out the final result
        final_value = cerebro.broker.getvalue()
        #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
        #print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))
        profit_dict[s] = (final_value - start_value) / start_value * 100
    
    profit_list.append((stock_c, sorted(profit_dict.items(), key=lambda x: x[1], reverse=True)[0:3]))
    time.sleep(0.2)

ZeroDivisionError: float division by zero

In [38]:
sorted(profit_list, key=lambda x: x[1][0][1], reverse=True)

[('601216.SH',
  [(('m', 'm'), 141.0),
   (('w', 'm'), 135.94),
   (('wr', 'm'), 86.83999999999999)]),
 ('600999.SH',
  [(('wr', 'b'), 64.86),
   (('r', 'b'), 53.879999999999995),
   (('r', 'w'), 46.68)]),
 ('601633.SH',
  [(('wr', 'w'), 44.76), (('wr', 'm'), 42.92), (('w', 'w'), 29.2)]),
 ('601377.SH',
  [(('wr', 'm'), 34.98),
   (('r', 'm'), 28.499999999999982),
   (('b', 'm'), 25.46)]),
 ('601601.SH',
  [(('r', 'b'), 29.78), (('wr', 'b'), 22.36), (('m', 'b'), 21.18)]),
 ('600989.SH',
  [(('wr', 'm'), 28.7), (('m', 'm'), 27.16), (('w', 'w'), 14.04)]),
 ('601577.SH',
  [(('wr', 'b'), 27.800000000000004),
   (('r', 'b'), 24.44),
   (('m', 'b'), 22.24)]),
 ('601066.SH', [(('wr', 'm'), 27.76), (('b', 'b'), 9.54), (('b', 'm'), 9.54)]),
 ('601628.SH',
  [(('m', 'b'), 27.200000000000003),
   (('w', 'b'), 26.840000000000003),
   (('b', 'b'), 25.580000000000002)]),
 ('601018.SH',
  [(('b', 'b'), 26.919999999999998),
   (('b', 'w'), 21.16),
   (('b', 'r'), 21.04)]),
 ('601336.SH',
  [(('r', 'b

In [33]:
sorted(profit_list, key=lambda x: x[1][0][1], reverse=True)

[('600760.SH',
  [(('wr', 'm'), 62.53999999999999), (('b', 'w'), 61.8), (('m', 'w'), 61.8)]),
 ('600196.SH',
  [(('m', 'm'), 52.44), (('b', 'm'), 5.6000000000000005), (('r', 'b'), 3.2)]),
 ('600893.SH',
  [(('wr', 'm'), 50.63999999999999),
   (('m', 'm'), 48.32),
   (('b', 'm'), 33.32)]),
 ('002415.SZ',
  [(('r', 'w'), 41.88),
   (('r', 'm'), 39.839999999999996),
   (('r', 'r'), 35.46)]),
 ('300059.SZ',
  [(('wr', 'm'), 40.36), (('wr', 'w'), 35.74), (('wr', 'b'), 34.88)]),
 ('002493.SZ',
  [(('r', 'm'), 38.56),
   (('r', 'w'), 36.919999999999995),
   (('r', 'r'), 30.84)]),
 ('002008.SZ',
  [(('wr', 'm'), 37.96),
   (('wr', 'b'), 29.12),
   (('wr', 'r'), 27.700000000000003)]),
 ('600606.SH',
  [(('wr', 'b'), 36.16),
   (('r', 'b'), 26.779999999999998),
   (('m', 'b'), 26.44)]),
 ('600741.SH',
  [(('wr', 'b'), 35.480000000000004),
   (('wr', 'w'), 28.98),
   (('wr', 'wr'), 17.78)]),
 ('002294.SZ',
  [(('wr', 'm'), 34.8), (('wr', 'w'), 34.4), (('m', 'm'), 33.44)]),
 ('002142.SZ',
  [(('wr